In [3]:
from scipy.special import j1
from scipy.optimize import brentq
from sage.all import *
import struct

DR = RealField(53)

DD = RealField(157)

def double_to_hex(f):
    # Converts Python float (f64) to hex string
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def format_dyadic_hex(value):
    l = hex(value)[2:]
    n = 8
    x = [l[i:i + n] for i in range(0, len(l), n)]
    return "0x" + "_".join(x) + "_u128"

def print_dyadic(value):
    (s, m, e) = RealField(128)(value).sign_mantissa_exponent();
    print("DyadicFloat128 {")
    print(f"    sign: DyadicSign::{'Pos' if s >= 0 else 'Neg'},")
    print(f"    exponent: {e},")
    print(f"    mantissa: {format_dyadic_hex(m)},")
    print("},")


In [28]:
def build_sollya_script(idx):
    return f"""
d = [{idx}/16, {idx + 1}/16];

f = 1/erf(x);
Q = fpminimax(f, [|0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24|], [|107, 107, 107, 107, 107, 107, D...|], d);

for i from 0 to degree(Q) by 2 do {{
    write(coeff(Q, i)) >> "coefficients.txt";
    write("\\n") >> "coefficients.txt";
}};
"""

def load_coefficients(filename):
    with open(filename, "r") as f:
        return [RealField(500)(line.strip()) for line in f if line.strip()]

def call_sollya_on_interval(idx):
    sollya_script = build_sollya_script(idx)
    with open("tmp_interval.sollya", "w") as f:
        f.write(sollya_script)
    import subprocess
    if os.path.exists("coefficients.txt"):
        os.remove("coefficients.txt")
    try:
        result = subprocess.run(
            ["sollya", "tmp_interval.sollya"],
            check=True,
            capture_output=True,
            text=True
        )
    except subprocess.CalledProcessError as e:
        return

def print_coeffs(poly):
    print("[")
    for i in range(len(poly)):
        coeff = poly[i]
        v_str = ""
        if i == 0:
            (lo, hi) = split_double_double(coeff)
            v_str += double_to_hex(lo) + "," + double_to_hex(hi) + ","
        elif i == 1:
            (lo, hi) = split_double_double(coeff)
            v_str += double_to_hex(lo) + "," + double_to_hex(hi) + ","
        elif i == 2:
            (lo, hi) = split_double_double(coeff)
            v_str += double_to_hex(lo) + "," + double_to_hex(hi) + ","
        elif i == 3:
            (lo, hi) = split_double_double(coeff)
            v_str += double_to_hex(lo) + "," + double_to_hex(hi) + ","
        elif i == 4:
            (lo, hi) = split_double_double(coeff)
            v_str += double_to_hex(lo) + "," + double_to_hex(hi) + ","
        elif i == 5:
            (lo, hi) = split_double_double(coeff)
            v_str += double_to_hex(lo) + "," + double_to_hex(hi) + ","
        else:
            v_str += double_to_hex(coeff) + ","
        print(v_str)
    print("],")

print(f"pub(crate) static RERF_FAST: [[u64; 18]; 95] = [")
for i in range(1, 96):
    call_sollya_on_interval(i)
    coeffs = load_coefficients(f"coefficients.txt")
    print_coeffs(coeffs)
print("];")

pub(crate) static RERF_FAST: [[u64; 18]; 95] = [
];


In [29]:
def build_sollya_script(idx):
    return f"""
d = [{idx}/16, {idx + 1}/16];

f = x/erf(x);
Q = fpminimax(f, [|0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20|], [|107...|], d);

for i from 0 to degree(Q) by 2 do {{
    write(coeff(Q, i)) >> "coefficients.txt";
    write("\\n") >> "coefficients.txt";
}};
"""

def load_coefficients(filename):
    with open(filename, "r") as f:
        return [RealField(500)(line.strip()) for line in f if line.strip()]

def call_sollya_on_interval(idx):
    sollya_script = build_sollya_script(idx)
    with open("tmp_interval.sollya", "w") as f:
        f.write(sollya_script)
    import subprocess
    if os.path.exists("coefficients.txt"):
        os.remove("coefficients.txt")
    try:
        result = subprocess.run(
            ["sollya", "tmp_interval.sollya"],
            check=True,
            capture_output=True,
            text=True
        )
    except subprocess.CalledProcessError as e:
        return

def print_coeffs(poly):
    print("[")
    for i in range(len(poly)):
        coeff = poly[i]
        print_double_double("", coeff)
    print("],")

print(f"pub(crate) static RERF_HARD: [[(u64, u64); 11]; 96] = [")
for i in range(0, 96):
    call_sollya_on_interval(i)
    coeffs = load_coefficients(f"coefficients.txt")
    print_coeffs(coeffs)
print("];")

pub(crate) static RERF_HARD: [[(u64, u64); 11]; 96] = [
];
